     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 28.0 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125925 sha256=2a495e6942ea795dfa024cd40165a8b0698eba0cf862fd534efcee7e5cc9a76c
  Stored in directory: /DATA/anurag_2101ai04/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence_transformers
Note: you may need to restart the kernel to use updated packages.


### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import re
import os
import string
import numpy as np
import pandas as pd

from tensorflow.python.keras import backend as K
from tensorflow.keras.regularizers import l2

from tensorflow.keras.models import Model, load_model
from tensorflow.keras import optimizers, initializers
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.applications.resnet import ResNet152

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.preprocessing import image as keras_image
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

from tensorflow.compat.v1 import ConfigProto, InteractiveSession
config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.8
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

# from keras.initializers import Constant
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Concatenate, Embedding, LSTM, Dropout, Bidirectional, Layer, InputSpec, Dot, Activation, Multiply

2023-10-08 10:34:44.811664: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-08 10:34:44.849920: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-08 10:34:45.074579: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-08 10:34:45.079634: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-08 10:34:47.859331: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [3]:
from sentence_transformers import SentenceTransformer, util
from PIL import Image
sentence_model = SentenceTransformer('paraphrase-mpnet-base-v2')

/DATA/anurag_2101ai04/.venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Data Preprocessing

In [4]:
dataset = pd.read_excel("./Aspect_Complain_web_entity_new.xlsx")
#dataset.drop(columns = ['Complaint Overall', 'Emotion Overall', 'Sentiment Overall','Emotion.0', 'Emotion.1', 'Emotion.2', 'Emotion.3', 'Comments'], inplace = True)
#dataset = dataset.iloc[0:1096,:]
dataset.rename(columns = {' ': 'Domain'}, inplace = True)
dataset = dataset.drop('Unnamed: 0',axis = 1)
# dataset = dataset.drop('Unnamed: 0.1',axis = 1)
dataset.columns

Index(['Domain', 'Complaint/ Opinion', 'Over-all_Complaint Label', 'Aspects_1',
       'Complaint_1', 'Cause_1', 'Aspect_2', 'Complaint_2', 'Cause_2',
       'Aspect_3', 'Complaint_3', 'Cause_3', 'Complaint_Cause',
       'Severity level', 'Sentiment', 'Emotion', 'Image Link', 'Web Entity'],
      dtype='object')

In [23]:
dataset

,Domain,Complaint/ Opinion,Over-all_Complaint Label,Aspects_1,Complaint_1,Cause_1,Aspect_2,Complaint_2,Cause_2,Aspect_3,Complaint_3,Cause_3,Complaint_Cause,Severity level,Sentiment,Emotion,Image Link,Web Entity
0,Customer Service,https://consumercomplaintscourt.com/ifb-washin...,1,Technical Issue,1.0,IFB washing machine,NaN,NaN,NaN,NaN,NaN,NaN,IFB washing machine,accusation,negative,anger,../BART/BART/blank.png,[]
1,Retail Banking,https://consumercomplaintscourt.com/i-want-to-...,1,Banking,1.0,want to forclose my dhani one freedom card at ...,NaN,NaN,NaN,NaN,NaN,NaN,I want to forclose my dhani one freedom card a...,Non-Complaint,Neutral,other,../BART/BART/blank.png,[]
2,Credit card,@ICICIBank @ICICIBank_Care wrist service from...,1,Delay Response,1.0,not able to refresh the service.,NaN,NaN,NaN,NaN,NaN,NaN,not able to refresh the service.,Disapproval,negative,sadness,../BART/BART/blank.png,[]
3,Credit card,@ICICIBank_Care @amazonpay can you help in get...,1,General Query,1.0,getting Amazon pay ICICI credit card,miscellaneous information,NaN,NaN,NaN,NaN,NaN,can you help in getting Amazon pay ICICI credi...,No explicit reproach,neutral,other,../BART/BART/blank.png,[]
4,Credit card,@ICICIBank_Care @ICICIBank it's been more than...,1,Delay Response,1.0,my refund is not credited to my credit card,NaN,NaN,NaN,NaN,NaN,NaN,my refund is not credited to my credit card,No explicit reproach,negative,sadness,../BART/BART/blank.png,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6464,Transactional Deficiency,@Venmo Getting surprise #venmo is already a gi...,0,Feedback,0.0,no cause,NaN,NaN,NaN,NaN,NaN,NaN,no cause,Non-Complaint,positive,surprise,../BART/BART/blank.png,[]
6465,Transactional Deficiency,"Cool of @Venmo to send a ""Use us as a last res...",0,Feedback,0.0,no cause,NaN,NaN,NaN,NaN,NaN,NaN,no cause,Non-Complaint,neutral,other,../BART/BART/6465.png,['Multimedia' 'Screenshot' 'Online advertising...
6466,Transactional Deficiency,who else follows Jimmy Darts on TikTok/IG and ...,0,Feedback,0.0,no cause,NaN,NaN,NaN,NaN,NaN,NaN,no cause,Non-Complaint,neutral,other,../BART/BART/6466.png,['Line' 'Font' 'Text' 'Number' 'Product' 'Mete...
6467,Transactional Deficiency,"1⃣ Bring balance, not complexity. @cvspharmacy...",0,Feedback,0.0,no cause,NaN,NaN,NaN,NaN,NaN,NaN,no cause,Non-Complaint,positive,happiness,../BART/BART/6467.png,['YouTube' nan 'Video quality' 'Multimedia' 'T...


In [5]:
# imgs_path = '../BART/BART/'
# dir = os.listdir(imgs_path)
# #dir.sort()

In [ ]:
# il = []
# for each in dir:
#   x = each.split('.')
#   x = x[0]
#   il.append(int(x)-1)

In [ ]:
# dataset = dataset.iloc[il]
# dataset

,Domain,Complaint/ Opinion,Over-all_Complaint Label,Aspects_1,Complaint_1,Aspect_2,Complaint_2,Aspect_3,Complaint_3,Complaint_Cause,Severity level,Sentiment,Emotion,image_name
18,Credit Card,@ICICIBank @ICICIBank_Care @BankCare9 @RBI @RB...,1,payment issue,1.0,miscellaneous information,1.0,NaN,NaN,credit card technical error,Disapproval,negative,disgust,19.png
20,Credit Card,@ICICIBank_Care @ICICIBank \nHey ICICI Bank I ...,1,miscellaneous information,1.0,NaN,NaN,NaN,NaN,They have been asking me to share the OTP,Accusation,negative,fear,21.png
39,Credit Card,@BlueDart_ @BlueDartCares @ICICIBank @ICICIBan...,1,miscellaneous information,1.0,NaN,NaN,NaN,NaN,I have not received any card from Bluedart,No explicit reproach,negative,sadness,40.png
68,Credit Card,@AmazonHelp @amazonIN @ICICIBank_Care I have I...,1,delay response,1.0,NaN,NaN,NaN,NaN,I have ICICI bank Amazon credit card and i hav...,No explicit reproach,negative,sadness,69.png
98,Credit Card,@HDFCBank_Cares @RBI This is not fair because ...,1,delay response,1.0,NaN,NaN,NaN,NaN,such a big bank is failed update credit card a...,No explicit reproach,negative,sadness,99.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6462,Transactional Deficiency,https://twitter.com/DenreleObaa/status/1605642...,1,technical issues,1.0,fraud,1.0,NaN,NaN,I made this transaction today and I was debite...,Blame,negative,Sadness,6463.png
6464,Transactional Deficiency,https://t.co/D0YoUitSLE,1,fraud,1.0,NaN,NaN,NaN,NaN,credit card Amount deducted 17800 Totaly fraud...,Blame,negative,disgust,6465.png
6465,Transactional Deficiency,https://t.co/5z66STWNeX,1,technical issues,1.0,NaN,NaN,NaN,NaN,Facing issue with link,Non-Explicit,negative,Sadness,6466.png
6466,Transactional Deficiency,https://t.co/aVogy5QH6d,1,technical issues,1.0,NaN,NaN,NaN,NaN,Amazon team saying issue is from capital float...,accusation,negative,disgust,6467.png


In [ ]:
# #directory = dir[965:1275]
# #directory+=(dir[:965])
# directory = []
# for i in range(len(dir)):
#   x = imgs_path + '/' + dir[i]
#   directory.append(x)

# images_path = directory

In [5]:
images_path = dataset['Image Link']

#### Getting data in lists

In [6]:
def get_title_review_comb(X_reviews):
    title_review_comb = []
    for i in range(X_reviews.shape[0]):
        if(str(type(X_reviews.iloc[i,0])) == "<class 'str'>"):
          title_review_comb.append(str(X_reviews.iloc[i,0])+str(X_reviews.iloc[i,1]))
        else:
          # print(i)
          title_review_comb.append(X_reviews.iloc[i,1])

    text_reviews = []
    for i in range(X_reviews.shape[0]):
        words = re.split(r'\W+', title_review_comb[i])

        words = [word.lower() for word in words]
        text = ' '.join(words)
        words = text.split()
        text = ' '.join(words)
        text_reviews.append(text)

    return text_reviews

# def check_aspect_comp(aspect,complaint_label,j):
#   aspects = []
#   comp_labels = []
#   for i in [0,1,2]:
#     val1 = aspect[i][j]
#     val2 = complaint_label[i][j]
#     if(str(val1) == 'nan') or val1 == ' ':
#       continue
#     elif(val1 == 'fabric'):
#       aspects.append('quality')
#     else:
#       aspects.append(val1)
#     comp_labels.append(val2)

#   return aspects,comp_labels

def check_aspect_comp(aspect,complaint_label,j):
  asp = []
  comp_labels = []
  for i in [0,1,2]:
    val = aspect[i][j]
    val2 = complaint_label[i][j]
    if(str(val) != 'nan'):
      val = str(val).lower().strip()
    if(str(val) == 'nan'):
      continue
    elif val in ['freedback','feeedback','feedbck','feedaback','fedback','feedabck','feedback','personal opinion','personal opinion,','general opinion','advise','genera review','general review']:
      asp.append('review')
    elif val in ['miscelllaneous information','miscellaneous inforamation','miscellaneous inforation','misceallneous information','miscellaneous information, inflation','misellaneous infromation','miscelleneos information','miscellaneous information','miscellaneous','miscellameous information','miscellaneus information','misellaneous information','miscelaneous information','misceklaneous information','miscellaneous iformation','misceallaneous information','financial advertisement','financial advertiement','financial advertisemnt']:
      asp.append('misc_info')
    elif val in ['delay, response','delay resoponse','delay respose','delay response','dealy response','delay resonse','delay response,','delay reponse','apologise']:
      asp.append('provider_response')
    elif val in ['payment','paymet','payent issue','payment issue','payment issue, bank issue','payment failure','payment failure, miscellaneous information','banking','bankinhg','bank issue','net banking','bank issue, harassment','internet banking','refund issue']:
      asp.append('net_banking_issue')
    # elif val in ['banking','bankinhg','bank issue','net banking','bank issue, harassment','internet banking']:
    #   asp.append('banking_issue')
    elif val in ['haraasment','harrasment','harassment','harrashment','fruad','fraud','manipulation']:
      asp.append('consumer_safety')
    # elif val in ['fruad','fraud']:
    #   asp.append('fraud')
    elif val in ['trade balance','news','dept collection','debt collection','exports']:
      asp.append('financial_info')
    # elif val in ['inflation']:
    #   asp.append('inflation')
    elif val in ['crdentials','credentials','credentials issue','credential','cedentials','credential related','technicality','techinicality','technical issue']:
      asp.append('credential_error')
    elif val in ['finacial policy','financial policy','financial policies','financial poilicy','financial stability','financial gain','inflation']:
      asp.append('financial_situation')
    # elif val in ['apologise']:
    #   asp.append('apologise')
    # elif val in ['dept collection','debt collection']:
    #   asp.append('debt_collection')
    # elif val in ['equal taxation']:
    #   asp.append('equal_taxation')
    elif val in ['genearl query','general qurey','general query','genral query','genaral query','general qurery','tax increases','equal taxation']:
      asp.append('general_query')
    # elif val in ['personal opinion','personal opinion,','general opinion','advise','genera review','general review']:
    #   asp.append('personal_opinion')
    # elif val in ['financial stability']:
    #   asp.append('financial_stability')
    # elif val in ['financial gain']:
    #   asp.append('financial_gain')
    # elif val in ['news']:
    #   asp.append('news')
    # elif val in ['manipulation']:
    #   asp.append('manipulation')
    # elif val in ['financial advertisement','financial advertiement','financial advertisemnt']:
    #   asp.append('financial_advertisement')
    # elif val in ['exports']:
    #   asp.append('exports')
    # elif val in ['tax increases','equal taxation']:
    #   asp.append('tax_implication')
    # elif val in ['technicality','techinicality','technical issue']:
    #   asp.append('technical_issue')
    # elif val in ['debt collection']:
    #   asp.append('debt collection')
    # elif val in ['refund issue']:
    #   asp.append('refund_issue')
    # else:
    #   print(val)
    else:
      asp.append(val)
    comp_labels.append(val2)
  return asp,comp_labels


def get_labels(df):
    aspect = [list(df['Aspects_1']), list(df['Aspect_2']), list(df['Aspect_3'])]
    complaint_label = [list(df['Complaint_1']), list(df['Complaint_2']), list(df['Complaint_3'])]
    comp_labels = []
    aspects = []

    for i in range(df.shape[0]):
        out = check_aspect_comp(aspect,complaint_label,i)
        aspects.append(out[0])
        comp_labels.append(out[1])

    return aspects, comp_labels

def get_web_entities(dataset):
    web_entities = []
    for mul_entities in list(dataset['Web_entities']):
      lables_list = (re.sub("[^a-zA-Z0-9,.)]", " ", mul_entities)).split(',')
      # print(lables_list)
      lables_list_mod = []
      for label in lables_list:
        label_mod = " ".join(label.split())
        # print(label_mod)
        if(label_mod == ''):
          continue
        lables_list_mod.append(label_mod.lower())

      web_entities.append(lables_list_mod)

    return web_entities


text_review = get_title_review_comb(dataset)
#web_entities = get_web_entities(dataset)
aspect_labels, complaint_labels = get_labels(dataset)

print(len(aspect_labels))
print(len(complaint_labels))
print(len(text_review))
#print(len(web_entities))
print(len(images_path))

6469
6469
6469
6469


In [26]:
# print(len(aspect_labels))
# print(len(complaint_labels))
# print(len(text_review))
# #print(len(web_entities))
print((images_path))

0       ../BART/BART/blank.png
1       ../BART/BART/blank.png
2       ../BART/BART/blank.png
3       ../BART/BART/blank.png
4       ../BART/BART/blank.png
                 ...          
6464    ../BART/BART/blank.png
6465     ../BART/BART/6465.png
6466     ../BART/BART/6466.png
6467     ../BART/BART/6467.png
6468     ../BART/BART/6468.png
Name: Image Link, Length: 6469, dtype: object


In [27]:
text_review

['customer servicehttps consumercomplaintscourt com ifb washing machine 2',
 'retail banking https consumercomplaintscourt com i want to forclose my dhani one freedom card at minimum value',
 'credit card icicibank icicibank_care wrist service from any bank credit card sicne yesterday more than 10 calls to customer care which take 15 mins to connect but still they are not able to refresh the service pathetic',
 'credit card icicibank_care amazonpay can you help in getting amazon pay icici credit card',
 'credit card icicibank_care icicibank it s been more than 4 months and my refund is not credited to my credit card and now you people want still more time what hell is going on',
 'credit card rbi icici bank has made a charge on my credit card that was closed 10 years ago their customer care is telling me it s due to some rbi rule but are not providing me clear details inspite of several calls and tweets icicibank_care icicibank nsitharaman finminindia',
 'credit card icicibank_care tod

### This is the code to convert multiple aspects to group of single aspect

In [28]:
review_aspect_merged = []
images_path_new = []
comp_labels = []
for r,a,c,i in zip(text_review, aspect_labels, complaint_labels, images_path):
    for aspect,complaint in zip(a,c):
        r_a = aspect+' [SEP] '+r
        review_aspect_merged.append(r_a)
        comp_labels.append(complaint)
        images_path_new.append(i)

print(len(review_aspect_merged))
print(len(images_path_new))
print(len(comp_labels))

7621
7621
7621


In [29]:
# First Split for Train and Test
text_train,text_test, img_train,img_test, y1_train,y1_test = train_test_split(np.array(text_review,dtype=object), np.array(images_path,dtype=object),

                                                                                                           np.array(aspect_labels,dtype=object), test_size=0.1, random_state=42, shuffle=True)
# Next split Train in to training and validation
text_tr,text_val, img_tr,img_val, y1_tr,y1_val = train_test_split(text_train, img_train, y1_train, test_size=0.2, random_state = 42, shuffle=True)


# print(web_ent_tr.shape)
# print(web_ent_test.shape)
# print(web_ent_val.shape)

In [30]:
review_aspect_merged[58]

'misc_info [SEP] credit card icicibank icicibank_care icici_direct please tell me what i have to do if i get a credit card statement that i dont own and why you assign relationship manager when they don t help or care for your customers icici icicicare fraud'

#### Encoding Text and Images

In [31]:
sentence_model.max_seq_length = 512
review_embeddings = []
for review in review_aspect_merged:
  text_emb = sentence_model.encode(review)
  review_embeddings.append(text_emb)

In [32]:
def image_preprocessing(path):
    img = keras_image.load_img(path, target_size=(224, 224), interpolation='bicubic')
    img = keras_image.img_to_array(img)
    img = img//255.0
    return img

images = []
for img_path in images_path:
  img = image_preprocessing(img_path)
  images.append(img)

images_list_new = []
for a,i in zip(aspect_labels, images):
    for aspect in a:
        images_list_new.append(i)

In [33]:
import tensorflow as tf
comp_labels = list(pd.Series(comp_labels).fillna(0.))
complaint_labels_reshaped = tf.keras.utils.to_categorical(comp_labels)
review_embeddings = np.array(review_embeddings).reshape(-1, 1, 768)
img_converted = tf.convert_to_tensor([tf.convert_to_tensor(i, dtype=tf.float32) for i in images_list_new], dtype = tf.float32)

In [34]:
complaint_labels_reshaped.shape

(7621, 2)

In [35]:
review_embeddings.shape

(7621, 1, 768)

#### Splitting dataset

In [36]:
RANDOM_SEED = 42
# First Split for Train and Test
img_train,img_test, x_train,x_test, y_train,y_test = train_test_split( np.array(img_converted),
                                                              np.array(review_embeddings), np.array(complaint_labels_reshaped),
                                                              test_size=0.1, random_state=RANDOM_SEED,shuffle=True)

# Next split Train in to training and validation
img_tr,img_val, x_tr,x_val, y_tr,y_val = train_test_split(img_train, x_train, y_train, test_size=0.2,
                                                          random_state = RANDOM_SEED,shuffle=True)

print(img_tr.shape)
print(img_test.shape)
print(img_val.shape)
print(x_tr.shape)
print(x_test.shape)
print(x_val.shape)
print(y_tr.shape)
print(y_test.shape)
print(y_val.shape)

(5486, 224, 224, 3)
(763, 224, 224, 3)
(1372, 224, 224, 3)
(5486, 1, 768)
(763, 1, 768)
(1372, 1, 768)
(5486, 2)
(763, 2)
(1372, 2)


In [37]:
# img_tr = tf.convert_to_tensor(img_tr, dtype=tf.float32)
# img_test = tf.convert_to_tensor(img_test, dtype=tf.float32)
# img_val = tf.convert_to_tensor(img_val, dtype=tf.float32)
print(img_tr.shape)
print(img_test.shape)
print(img_val.shape)

(5486, 224, 224, 3)
(763, 224, 224, 3)
(1372, 224, 224, 3)


### Attention

In [38]:
class AttLayer(Layer):
    """
    Computes a weighted average of the different channels across timesteps.
    Uses 1 parameter pr. channel to compute the attention value for a single timestep.
    """

    def __init__(self, return_attention=False, **kwargs):
        self.init = initializers.get('uniform')
        self.supports_masking = True
        self.return_attention = return_attention
        super(AttLayer, self).__init__(** kwargs)

    def get_config(self):
        config = {
            'return_attention': self.return_attention,
        }
        base_config = super(AttLayer, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def build(self, input_shape):
        self.input_spec = [InputSpec(ndim=3)]
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[2], 1),
                                 name='{}_W'.format(self.name),
                                 initializer=self.init)
        self._trainable_weights = [self.W]
        super(AttLayer, self).build(input_shape)

    def call(self, x, mask=None):
        # computes a probability distribution over the timesteps
        # uses 'max trick' for numerical stability
        # reshape is done to avoid issue with Tensorflow
        # and 1-dimensional weights
        logits = K.dot(x, self.W)
        x_shape = K.shape(x)
        logits = K.reshape(logits, (x_shape[0], x_shape[1]))
        ai = K.exp(logits - K.max(logits, axis=-1, keepdims=True))

        # masked timesteps have zero weight
        if mask is not None:
            mask = K.cast(mask, K.floatx())
            ai = ai * mask
        att_weights = ai / (K.sum(ai, axis=1, keepdims=True) + K.epsilon())
        weighted_input = x * K.expand_dims(att_weights)
        result = K.sum(weighted_input, axis=1)
        if self.return_attention:
            return [result, att_weights]
        return result

    def get_output_shape_for(self, input_shape):
        return self.compute_output_shape(input_shape)

    def compute_output_shape(self, input_shape):
        output_len = input_shape[2]
        if self.return_attention:
            return [(input_shape[0], output_len), (input_shape[0], input_shape[1])]
        return (input_shape[0], output_len)

    def compute_mask(self, input, input_mask=None):
        if isinstance(input_mask, list):
            return [None] * len(input_mask)
        else:
            return None

### Modelling

In [39]:
resnet152 = ResNet152(input_shape = (224, 224, 3), include_top = False, weights = 'imagenet')
for layer in resnet152.layers:
    layer.trainable = False
input_1 = Input(shape=(1, 768), dtype='float32')
image_input = Input(shape=(224, 224, 3), name='Images_input')
resnet152_output = resnet152(image_input)
pooling_layer = tf.keras.layers.GlobalAveragePooling2D()(resnet152_output)
img_repr = (tf.reshape(pooling_layer, [-1, 1, 2048]))
concatenated_features = Concatenate(axis = -1)([input_1, img_repr])
Bi_lstm = Bidirectional(tf.keras.layers.GRU(256, return_sequences = True, dropout=0.3, recurrent_dropout=0.2, kernel_regularizer = l2(0.000001)))(concatenated_features)
Bi_lstm = AttLayer()(Bi_lstm)
out_aspects = Dropout(0.3)(Bi_lstm)
out_aspects = Dense(128, activation = 'relu')(out_aspects)
out_aspects = Dropout(0.2)(out_aspects)
output = Dense(2, activation = 'softmax')(out_aspects)

MM_model = Model([input_1, image_input], output)
MM_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 Images_input (InputLayer)   [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 resnet152 (Functional)      (None, 7, 7, 2048)           5837094   ['Images_input[0][0]']        
                                                          4                                       
                                                                                                  
 global_average_pooling2d (  (None, 2048)                 0         ['resnet152[0][0]']           
 GlobalAveragePooling2D)                                                                          
                                                                                              

### Training

In [40]:
m = 'accuracy'
MM_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
                  loss = 'categorical_crossentropy',
                  loss_weights = 0.5,
                  metrics = m)

earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=13, verbose=1, restore_best_weights=True)

MM_model.fit(
    [x_tr, img_tr], y_tr,
    batch_size=16,
    epochs=15,
    validation_data = ([x_val, img_val], y_val),
    steps_per_epoch = x_tr.shape[0] // 16,
    validation_steps = x_val.shape[0] // 16,
    shuffle=True,
    verbose=1,
    callbacks=[earlystopping]
)


Epoch 1/15
342/342 [==============================] - 302s 849ms/step - loss: 0.1800 - accuracy: 0.8512 - val_loss: 0.0893 - val_accuracy: 0.9338
Epoch 2/15
342/342 [==============================] - 294s 860ms/step - loss: 0.0824 - accuracy: 0.9468 - val_loss: 0.0620 - val_accuracy: 0.9588
Epoch 3/15
342/342 [==============================] - 292s 855ms/step - loss: 0.0733 - accuracy: 0.9510 - val_loss: 0.0593 - val_accuracy: 0.9625
Epoch 4/15
342/342 [==============================] - 292s 854ms/step - loss: 0.0645 - accuracy: 0.9567 - val_loss: 0.0607 - val_accuracy: 0.9625
Epoch 5/15
342/342 [==============================] - 268s 784ms/step - loss: 0.0611 - accuracy: 0.9618 - val_loss: 0.0531 - val_accuracy: 0.9684
Epoch 6/15
342/342 [==============================] - 278s 812ms/step - loss: 0.0580 - accuracy: 0.9640 - val_loss: 0.0523 - val_accuracy: 0.9706
Epoch 7/15
342/342 [==============================] - 295s 862ms/step - loss: 0.0579 - accuracy: 0.9636 - val_loss: 0.0543 -

In [41]:
MM_model.save('best_complaint_detection_label.h5')

/DATA/anurag_2101ai04/.venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Testing

In [1]:
MM_model = load_model('best_complaint_detection.h5', custom_objects={'AttLayer': AttLayer})

NameError: name 'load_model' is not defined

In [42]:
complaint_labels_predicted = MM_model.predict([x_test, img_test])
complaint_labels_pred = np.argmax(complaint_labels_predicted, axis = -1)
complaint_labels_predicted

24/24 [==============================] - 29s 1s/step


array([[2.2002889e-04, 9.9978000e-01],
       [2.8936097e-02, 9.7106385e-01],
       [5.0115730e-03, 9.9498838e-01],
       ...,
       [1.7994786e-04, 9.9982005e-01],
       [2.7561208e-04, 9.9972433e-01],
       [9.9997187e-01, 2.8090284e-05]], dtype=float32)

In [43]:
complaint_labels_true = []
for inst in y_test:
  if inst[0] == 1:
    complaint_labels_true.append(0)
  elif inst[1] == 1:
    complaint_labels_true.append(1)
complaint_labels_true = np.array(complaint_labels_true)

In [44]:
accuracy_score(complaint_labels_true, complaint_labels_pred)

0.9580602883355177

In [45]:
complaint_labels_true.shape

(763,)

In [46]:
complaint_labels_pred.shape

(763,)

In [47]:
from sklearn.metrics import classification_report
print('Complaint(1)/Non-complaint(0) Report')
print(classification_report(complaint_labels_true, complaint_labels_pred))

Complaint(1)/Non-complaint(0) Report
              precision    recall  f1-score   support

           0       0.99      0.91      0.95       324
           1       0.94      0.99      0.96       439

    accuracy                           0.96       763
   macro avg       0.96      0.95      0.96       763
weighted avg       0.96      0.96      0.96       763



In [28]:
MM_model.save('best_complaint_detection.h5')
MM_model.save('best_complaint_detection.keras')

/DATA/anurag_2101ai04/.venv/lib/python3.8/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Finding the domain wise metrics (Ignore remaining part)

In [27]:
# Create a counter for the number of unique domain in dataset['Domain'] strip it and lower the case
domain = []
for i in dataset['Domain']:
    domain.append(i.strip().lower())
domain = list(domain)
from collections import Counter
domain_counter = Counter(domain)
print(domain_counter)

Counter({'transactional deficiency': 1546, 'investments': 1049, 'debit card': 853, 'customer service': 760, 'economy': 719, 'credit card': 666, 'loan': 386, 'financial policies': 327, 'retail banking': 110, 'salary': 53})


In [28]:
# convert each of the domain in dataset['Domain'] to lower case and strip it inplace in the original dataset
dataset['Domain'] = dataset['Domain'].str.lower()
dataset['Domain'] = dataset['Domain'].str.strip()

In [29]:
dataset['Domain'].unique()

array(['customer service', 'retail banking', 'credit card', 'debit card',
       'economy', 'financial policies', 'investments', 'loan', 'salary',
       'transactional deficiency'], dtype=object)

In [39]:
# Make seperate predections for all the domains and generate seperate classification reports for each of the domains
for domain in dataset['Domain'].unique():
    print('Domain:', domain)
    domain_df = dataset[dataset['Domain'] == domain]
    domain_df.reset_index(inplace=True)
    domain_df.drop(columns=['index'], inplace=True)
    print(domain_df.shape)
    domain_text_reviews = get_title_review_comb(domain_df)
    domain_aspect_labels, domain_complaint_labels = get_labels(domain_df)
    domain_review_aspect_merged = []
    domain_images_path_new = []
    domain_comp_labels = []
    for r,a,c,i in zip(domain_text_reviews, domain_aspect_labels, domain_complaint_labels, images_path):
        for aspect,complaint in zip(a,c):
            r_a = aspect+' [SEP] '+r
            domain_review_aspect_merged.append(r_a)
            domain_comp_labels.append(complaint)
            domain_images_path_new.append(i)
    domain_review_embeddings = []
    for review in domain_review_aspect_merged:
        text_emb = sentence_model.encode(review)
        domain_review_embeddings.append(text_emb)
    domain_images = []
    for img_path in domain_images_path_new:
        img = image_preprocessing(img_path)
        domain_images.append(img)
    domain_images_list_new = []
    for a,i in zip(domain_aspect_labels, domain_images):
        for aspect in a:
            domain_images_list_new.append(i)
    domain_comp_labels = list(pd.Series(domain_comp_labels).fillna(0.))
    domain_complaint_labels_reshaped = tf.keras.utils.to_categorical(domain_comp_labels)
    domain_review_embeddings = np.array(domain_review_embeddings).reshape(-1, 1, 768)
    domain_img_converted = tf.convert_to_tensor([tf.convert_to_tensor(i, dtype=tf.float32) for i in domain_images_list_new], dtype = tf.float32)
    # domain_img_train,domain_img_test, domain_x_train,domain_x_test, domain_y_train,domain_y_test = train_test_split( np.array(domain_img_converted),
    #                                                                                                                           np.array(domain_review_embeddings), np.array(domain_complaint_labels_reshaped),
    #                                                                                                                           test_size=0.1, random_state=RANDOM_SEED,shuffle=True)
    # domain_img_tr,domain_img_val, domain_x_tr,domain_x_val, domain_y_tr,domain_y_val = train_test_split(domain_img_train, domain_x_train, domain_y_train, test_size=0.2,
    #                                                                                                     random_state = RANDOM_SEED,shuffle=True)
    domain_complaint_labels_predicted = MM_model.predict([domain_review_embeddings, domain_img_converted])    
    domain_label_pred = np.argmax(domain_complaint_labels_predicted, axis = -1)
    complaint_domain_labels_true = []
    for inst in domain_complaint_labels_reshaped:
        if inst[0] == 1:
            complaint_domain_labels_true.append(0)
        elif inst[1] == 1:
            complaint_domain_labels_true.append(1)
    complaint_domain_labels_true = np.array(complaint_domain_labels_true)
    print('Complaint(1)/Non-complaint(0) Report')
    print(complaint_domain_labels_true.shape)
    print(complaint_labels_pred.shape)
    print(accuracy_score(complaint_domain_labels_true, complaint_labels_pred))
    print(classification_report(complaint_domain_labels_true, complaint_labels_pred))

Domain: customer service
(760, 18)


/tmp/ipykernel_6928/888047093.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domain_df.drop(columns=['index'], inplace=True)


1/1 [==============================] - 0s 464ms/step
Complaint(1)/Non-complaint(0) Report
(1024,)
(761,)


ValueError: Found input variables with inconsistent numbers of samples: [1024, 1]

In [42]:
domain_df = dataset[dataset['Domain'] == 'customer service']
domain_df.reset_index(inplace=True)
domain_df.drop(columns=['index'], inplace=True)
print(domain_df.shape)
domain_text_reviews = get_title_review_comb(domain_df)
domain_aspect_labels, domain_complaint_labels = get_labels(domain_df)
domain_image_path = [item for item in dataset[dataset['Domain'] == 'customer service']['Image Link']]

(760, 18)


/tmp/ipykernel_6928/2021918148.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  domain_df.drop(columns=['index'], inplace=True)


In [41]:
domain_review_aspect_merged = []
domain_images_path_new = []
domain_comp_labels = []
for r,a,c,i in zip(domain_text_reviews, domain_aspect_labels, domain_complaint_labels, images_path):
    for aspect,complaint in zip(a,c):
        r_a = aspect+' [SEP] '+r
        domain_review_aspect_merged.append(r_a)
        domain_comp_labels.append(complaint)
        domain_images_path_new.append(i)

In [43]:
domain_review_embeddings = []
for review in domain_review_aspect_merged:
    text_emb = sentence_model.encode(review)
    domain_review_embeddings.append(text_emb)
domain_images = []
for img_path in domain_image_path:
    img = image_preprocessing(img_path)
    domain_images.append(img)
domain_images_list_new = []
for a,i in zip(domain_aspect_labels, domain_images):
    for aspect in a:
        domain_images_list_new.append(i)

In [44]:
domain_comp_labels = list(pd.Series(domain_comp_labels).fillna(0.))
domain_complaint_labels_reshaped = tf.keras.utils.to_categorical(domain_comp_labels)
domain_review_embeddings = np.array(domain_review_embeddings).reshape(-1, 1, 768)
domain_img_converted = tf.convert_to_tensor([tf.convert_to_tensor(i, dtype=tf.float32) for i in domain_images_list_new], dtype = tf.float32)

In [45]:
domain_complaint_labels_predicted = MM_model.predict([domain_review_embeddings, domain_img_converted])    
domain_label_pred = np.argmax(domain_complaint_labels_predicted, axis = -1)
complaint_domain_labels_true = []
for inst in domain_complaint_labels_reshaped:
    if inst[0] == 1:
        complaint_domain_labels_true.append(0)
    elif inst[1] == 1:
        complaint_domain_labels_true.append(1)
complaint_domain_labels_true = np.array(complaint_domain_labels_true)

32/32 [==============================] - 41s 1s/step


In [46]:
print(complaint_domain_labels_true.shape)
print(complaint_labels_pred.shape)

(1024,)
(761,)


In [47]:
accuracy_score(complaint_domain_labels_true, complaint_labels_pred)

ValueError: Found input variables with inconsistent numbers of samples: [1024, 761]

In [ ]:
review_aspect_merged[36:38]

['banking [SEP] customer service theofficialsbi horrible app not able to register again after update home branch doesn t know how to help nri customers can t come down in person so it s easy to neglect have written lot of emails already to nri customer care pathetic service and response no one has solution https t co 9ulqni2ukn',
 'miscellaneous information [SEP] customer service theofficialsbi horrible app not able to register again after update home branch doesn t know how to help nri customers can t come down in person so it s easy to neglect have written lot of emails already to nri customer care pathetic service and response no one has solution https t co 9ulqni2ukn']

In [ ]:
x,y = 37,38
pred = MM_model.predict([np.array(review_embeddings[x-1:y]), np.array(img_converted[x-1:y])])

1/1 [==============================] - 1s 701ms/step


In [ ]:
pred

array([[0.00210179, 0.9978982 ],
       [0.00545172, 0.99454826]], dtype=float32)

In [ ]:
images_path_new[36:38]

['/content/drive/MyDrive/final_internship/about/new_img/719.png',
 '/content/drive/MyDrive/final_internship/about/new_img/719.png']

In [ ]:
review_aspect_merged[40:43]

['delay response [SEP] customer service theofficialsbi 7th reminder and 100 days but no response from sbi lho mumbai after repeatedly followup what is the deadline to resolve fraudulent transaction reported in july 2021 6 months before local branch submitted all documents as requested by lho rbi mahacyber1 theofficialsbi https t co xett21vu34',
 'miscellaneous information [SEP] customer service theofficialsbi 7th reminder and 100 days but no response from sbi lho mumbai after repeatedly followup what is the deadline to resolve fraudulent transaction reported in july 2021 6 months before local branch submitted all documents as requested by lho rbi mahacyber1 theofficialsbi https t co xett21vu34',
 'fraud [SEP] customer service theofficialsbi 7th reminder and 100 days but no response from sbi lho mumbai after repeatedly followup what is the deadline to resolve fraudulent transaction reported in july 2021 6 months before local branch submitted all documents as requested by lho rbi mahacyb

In [ ]:
x,y = 41,43
pred = MM_model.predict([np.array(review_embeddings[x-1:y]), np.array(img_converted[x-1:y])])

1/1 [==============================] - 1s 728ms/step


In [ ]:
pred

array([[1.5941956e-04, 9.9984050e-01],
       [2.1856482e-04, 9.9978143e-01],
       [2.4461246e-04, 9.9975544e-01]], dtype=float32)

In [ ]:
images_path_new[40:43]

['/content/drive/MyDrive/final_internship/about/new_img/748.png',
 '/content/drive/MyDrive/final_internship/about/new_img/748.png',
 '/content/drive/MyDrive/final_internship/about/new_img/748.png']

In [ ]:
review_aspect_merged[51:53]

['delay response [SEP] customer service icicibank icicibank_care icici_direct icicilombard icicidirectcare i am still trying to activate my account but no response ony new date for resolution worst services they provide https t co sb6q5onqjg',
 'credentials issue [SEP] customer service icicibank icicibank_care icici_direct icicilombard icicidirectcare i am still trying to activate my account but no response ony new date for resolution worst services they provide https t co sb6q5onqjg']

In [ ]:
x,y = 52,53
pred = MM_model.predict([np.array(review_embeddings[x-1:y]), np.array(img_converted[x-1:y])])
pred

1/1 [==============================] - 0s 37ms/step


array([[6.8356359e-04, 9.9931645e-01],
       [1.7253376e-03, 9.9827468e-01]], dtype=float32)

In [ ]:
images_path_new[51:53]

['/content/drive/MyDrive/final_internship/about/new_img/811.png',
 '/content/drive/MyDrive/final_internship/about/new_img/811.png']

In [ ]:
review_aspect_merged[183:184]

['financial advertisement [SEP] debit cardskating in the park at the hsvmuseumart is back during december redstonefcu members can receive 20 off one admission to skatinginthepark when using their redstone debit or credit card discount must be issued at the time of purchase ihearthsv downtownhsv https t co 1xqq9cp6pe']

In [ ]:
x,y = 184,184
pred = MM_model.predict([np.array(review_embeddings[x-1:y]), np.array(img_converted[x-1:y])])
pred

1/1 [==============================] - 0s 53ms/step


array([[0.9977882 , 0.00221182]], dtype=float32)

In [ ]:
images_path_new[183]

'/content/drive/MyDrive/final_internship/about/new_img/1675.png'

In [ ]:
review_aspect_merged[132:133]

['miscellaneous information [SEP] customer service amazonhelp no i am facing issue with link and other thing is now i am getting this please give me the actual solution instead of routing customer care very bad service not only courier but cc counterpart as well need resolution asap https t co 5z66stwnex']

In [ ]:
txt = 'delay response [SEP] customer service amazonhelp no i am facing issue with link and other thing is now i am getting this please give me the actual solution instead of routing customer care very bad service not only courier but cc counterpart as well need resolution asap https t co 5z66stwnex'

In [ ]:
txt = sentence_model.encode(txt)

In [ ]:
txt

array([-0.01375236, -0.15847366, -0.02591123,  0.01127579,  0.00676969,
        0.05496904, -0.01739257,  0.06125331,  0.13632137,  0.05493233,
        0.23480764, -0.04989847,  0.11362431,  0.06126961, -0.1099728 ,
       -0.01826554,  0.06797828, -0.03413766, -0.00539552,  0.13346863,
        0.06067589,  0.01323043,  0.08538115, -0.07117473, -0.04977993,
        0.00208392,  0.00344402, -0.10125175, -0.07341199,  0.02573583,
        0.17463356, -0.04612756,  0.01081635, -0.03938123,  0.0372438 ,
        0.038663  ,  0.16234683,  0.05569602, -0.14919193, -0.08160208,
       -0.08490689, -0.23658746,  0.01114135,  0.02023952, -0.07153404,
        0.18757914, -0.1139606 ,  0.1460458 ,  0.24603271,  0.14683019,
       -0.04488626,  0.11557961,  0.02549067,  0.03434632,  0.20532134,
       -0.18821375, -0.12818265, -0.11616487,  0.12338983,  0.06927601,
       -0.06292785, -0.07287945,  0.09222902, -0.05829621, -0.1084304 ,
       -0.16100505, -0.02339886, -0.04504775, -0.18554154,  0.00

In [ ]:
review_embeddings[132] = txt

In [ ]:
x,y = 133,133
pred = MM_model.predict([np.array(review_embeddings[x-1:y]), np.array(img_converted[x-1:y])])
pred

1/1 [==============================] - 0s 51ms/step


array([[0.00118327, 0.9988167 ]], dtype=float32)

In [ ]:
images_path_new[132]

'/content/drive/MyDrive/final_internship/about/new_img/1350.png'